In [ ]:
#Example
from queue import PriorityQueue

graph = {
    'A': [('B', 5), ('C', 8)],
    'B': [('D', 10)],
    'C': [('E', 3)],
    'D': [('F', 7)],
    'E': [('F', 2)],
    'F': []
}

def bfs(graph, start, goal):
    visited = set()
    pq = PriorityQueue()
    pq.put((0, start))

    while not pq.empty():
        cost, node = pq.get()
        print(f"Priority Queue: {pq.queue} Visiting {node} with cost {cost}")

        if node in visited:
            continue

        print(f"Current Node: {node}")
        visited.add(node)

        if node == goal:
            print("\nGoal found!")
            return True

        for neighbor, weight in graph[node]:
            if neighbor not in visited:
                pq.put((cost + weight, neighbor))

    print("\nGoal not found")
    return False

print("UCS Path:")
bfs(graph, 'A', 'F')


UCS Path:
Priority Queue: [] Visiting A with cost 0
Current Node: A
Priority Queue: [(8, 'C')] Visiting B with cost 5
Current Node: B
Priority Queue: [(15, 'D')] Visiting C with cost 8
Current Node: C
Priority Queue: [(15, 'D')] Visiting E with cost 11
Current Node: E
Priority Queue: [(15, 'D')] Visiting F with cost 13
Current Node: F

Goal found!


True

In [ ]:
#TASK 1

from queue import PriorityQueue
from itertools import permutations

class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.g = 0
        self.h = 0
        self.f = 0

    def __lt__(self, other):
        return self.f < other.f


def heuristic(current_pos, end_pos):
    return abs(current_pos[0] - end_pos[0]) + abs(current_pos[1] - end_pos[1])


def best_first_search(maze, start, end):
    rows, cols = len(maze), len(maze[0])
    start_node = Node(start)
    start_node.h = heuristic(start, end)
    start_node.f = start_node.h

    frontier = PriorityQueue()
    frontier.put(start_node)
    visited = set()

    while not frontier.empty():
        current_node = frontier.get()
        current_pos = current_node.position

        if current_pos == end:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]

        visited.add(current_pos)

        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            new_pos = (current_pos[0] + dx, current_pos[1] + dy)

            if (
                0 <= new_pos[0] < rows and 0 <= new_pos[1] < cols
                and maze[new_pos[0]][new_pos[1]] == 0
                and new_pos not in visited
            ):
                new_node = Node(new_pos, current_node)
                new_node.g = current_node.g + 1
                new_node.h = heuristic(new_pos, end)
                new_node.f = new_node.h

                frontier.put(new_node)
                visited.add(new_pos)

    return None


def find_shortest_path_covering_all_goals(maze, start, goal_points):
    all_points = [start] + list(goal_points)
    min_path = None
    min_distance = float('inf')

    for perm in permutations(goal_points):
        total_path = []
        total_distance = 0
        current_pos = start

        for goal in perm:
            path_segment = best_first_search(maze, current_pos, goal)
            if not path_segment:
                total_distance = float('inf')
                break
            total_path.extend(path_segment[:-1])
            total_distance += len(path_segment) - 1
            current_pos = goal

        if total_distance < min_distance:
            min_distance = total_distance
            min_path = total_path + [perm[-1]]

    return min_path


maze = [
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 1, 0, 0, 0, 0]
]

start = (0, 0)
goal_points = {(0,3),(3,5),(4, 4)}

path = find_shortest_path_covering_all_goals(maze, start, goal_points)

if path:
    print("Path found:", path)
else:
    print("No path found")


Path found: [(0, 0), (0, 1), (1, 1), (1, 2), (1, 3), (0, 3), (1, 3), (2, 3), (3, 3), (3, 4), (4, 4), (3, 4), (3, 5)]


In [5]:
#TASK 2
import random
import time

class DynamicAStar:
    def __init__(self, graph, heuristic, start, goal):
        self.graph = graph
        self.heuristic = heuristic
        self.start = start
        self.goal = goal
        self.frontier = [(start, heuristic[start])]
        self.visited = set()
        self.g_costs = {start: 0}
        self.came_from = {start: None}

    def update_edge_costs(self):
        node = random.choice(list(self.graph.keys()))
        if self.graph[node]:
            neighbor = random.choice(list(self.graph[node].keys()))
            new_cost = random.randint(1, 10)
            self.graph[node][neighbor] = new_cost
            self.graph[neighbor][node] = new_cost
            print(f"\nEdge cost updated: ({node} -> {neighbor}) : {new_cost}")

    def a_star_search(self):
        while self.frontier:
            self.frontier.sort(key=lambda x: x[1])
            current_node, _ = self.frontier.pop(0)

            if current_node in self.visited:
                continue

            print(current_node, end=" ")
            self.visited.add(current_node)

            if current_node == self.goal:
                path = []
                while current_node is not None:
                    path.append(current_node)
                    current_node = self.came_from[current_node]
                path.reverse()
                print(f"\nGoal found! Path: {path}, Cost: {self.g_costs[self.goal]}")
                return

            for neighbor, cost in self.graph[current_node].items():
                new_g_cost = self.g_costs[current_node] + cost
                f_cost = new_g_cost + self.heuristic[neighbor]

                if neighbor not in self.g_costs or new_g_cost < self.g_costs[neighbor]:
                    self.g_costs[neighbor] = new_g_cost
                    self.came_from[neighbor] = current_node
                    self.frontier.append((neighbor, f_cost))

            time.sleep(1)
            if random.random() < 0.4:
                self.update_edge_costs()

        print("\nGoal not found.")

graph = {
    'X': {'Y': 2, 'Z': 5},
    'Y': {'X': 2, 'W': 3, 'V': 7},
    'Z': {'X': 5, 'U': 4},
    'W': {'Y': 3, 'T': 6},
    'V': {'Y': 7, 'T': 1},
    'U': {'Z': 4, 'T': 3},
    'T': {'W': 6, 'V': 1, 'U': 3}
}

heuristic = {
    'X': 8, 'Y': 6, 'Z': 5,
    'W': 4, 'V': 3, 'U': 2, 'T': 0
}

start_node = 'X'
goal_node = 'T'

agent = DynamicAStar(graph, heuristic, start_node, goal_node)
print("\nRunning A* Search with Dynamic Edge Costs:")
agent.a_star_search()



Running A* Search with Dynamic Edge Costs:
X Y 
Edge cost updated: (V -> Y) : 6
W 
Edge cost updated: (X -> Y) : 1
Z T 
Goal found! Path: ['X', 'Y', 'W', 'T'], Cost: 11


In [6]:
#TASK 3
import random
import time

class DeliveryRouteGBFS:
    def __init__(self, graph, heuristic, time_windows, start, goal):
        self.graph = graph
        self.heuristic = heuristic
        self.time_windows = time_windows
        self.start = start
        self.goal = goal
        self.current_time = 0
        self.frontier = [(start, heuristic[start], self.time_windows[start][0])]
        self.visited = set()
        self.came_from = {start: None}
        self.delivery_times = {start: self.time_windows[start][0]}

    def greedy_bfs(self):
        while self.frontier:
            self.frontier.sort(key=lambda x: (x[1], x[2]))
            current_node, _, arrival_time = self.frontier.pop(0)

            if current_node in self.visited:
                continue

            print(f"Visiting: {current_node}, Arrival Time: {arrival_time}")
            self.visited.add(current_node)
            self.current_time = arrival_time

            if current_node == self.goal:
                path = []
                while current_node is not None:
                    path.append(current_node)
                    current_node = self.came_from[current_node]
                path.reverse()
                print(f"\nOptimal Delivery Route Found: {path}")
                return

            for neighbor, travel_time in self.graph[current_node].items():
                new_arrival_time = self.current_time + travel_time

                if neighbor not in self.visited and new_arrival_time <= self.time_windows[neighbor][1]:
                    self.came_from[neighbor] = current_node
                    self.frontier.append((neighbor, self.heuristic[neighbor], max(new_arrival_time, self.time_windows[neighbor][0])))
                    self.delivery_times[neighbor] = max(new_arrival_time, self.time_windows[neighbor][0])

        print("\nNo feasible delivery route found.")


graph = {
    'P': {'Q': 3, 'R': 2},
    'Q': {'S': 5, 'T': 4},
    'R': {'U': 1, 'V': 6},
    'S': {'W': 2},
    'T': {},
    'U': {'X': 7},
    'V': {},
    'W': {},
    'X': {}
}

heuristic = {
    'P': 8, 'Q': 7, 'R': 6,
    'S': 5, 'T': 9, 'U': 4,
    'V': 7, 'W': 3, 'X': 0
}

time_windows = {
    'P': (0, 12),
    'Q': (3, 10),
    'R': (2, 9),
    'S': (6, 14),
    'T': (7, 16),
    'U': (4, 11),
    'V': (5, 13),
    'W': (8, 15),
    'X': (9, 18)
}

start_node = 'P'
goal_node = 'X'

agent = DeliveryRouteGBFS(graph, heuristic, time_windows, start_node, goal_node)
print("\nRunning Greedy Best-First Search for Delivery Optimization:")
agent.greedy_bfs()



Running Greedy Best-First Search for Delivery Optimization:
Visiting: P, Arrival Time: 0
Visiting: R, Arrival Time: 2
Visiting: U, Arrival Time: 4
Visiting: X, Arrival Time: 11

Optimal Delivery Route Found: ['P', 'R', 'U', 'X']


In [8]:
#TASK 4
import heapq
import random
import time

class AStarTraffic:
    def __init__(self, graph, heuristic, start, goal):
        self.graph = graph
        self.heuristic = heuristic
        self.start = start
        self.goal = goal
        self.traffic_conditions = {edge: self.graph[edge[0]][edge[1]] for edge in self.get_edges()}

    def get_edges(self):
        edges = []
        for node, neighbors in self.graph.items():
            for neighbor in neighbors:
                edges.append((node, neighbor))
        return edges

    def update_traffic(self):
        for edge in self.traffic_conditions:
            self.traffic_conditions[edge] = max(1, self.graph[edge[0]][edge[1]] + random.randint(-1, 3))

    def a_star(self):
        frontier = []
        heapq.heappush(frontier, (0 + self.heuristic[self.start], 0, self.start))
        came_from = {self.start: None}
        g_costs = {self.start: 0}

        while frontier:
            _, current_g, current_node = heapq.heappop(frontier)

            if current_node == self.goal:
                path = []
                while current_node is not None:
                    path.append(current_node)
                    current_node = came_from[current_node]
                path.reverse()
                return path, g_costs[self.goal]

            if random.random() < 0.4:
                self.update_traffic()

            for neighbor in self.graph[current_node]:
                new_g_cost = current_g + self.traffic_conditions[(current_node, neighbor)]
                if neighbor not in g_costs or new_g_cost < g_costs[neighbor]:
                    g_costs[neighbor] = new_g_cost
                    came_from[neighbor] = current_node
                    f_cost = new_g_cost + self.heuristic[neighbor]
                    heapq.heappush(frontier, (f_cost, new_g_cost, neighbor))

        return None, float('inf')

graph = {
    'X': {'Y': 3, 'Z': 5},
    'Y': {'X': 3, 'W': 4, 'V': 8},
    'Z': {'X': 5, 'U': 2},
    'W': {'Y': 4, 'T': 7},
    'V': {'Y': 8, 'T': 3},
    'U': {'Z': 2, 'T': 1},
    'T': {'W': 7, 'V': 3, 'U': 1}
}

heuristic = {
    'X': 9, 'Y': 7, 'Z': 6,
    'W': 5, 'V': 4, 'U': 3, 'T': 0
}

start_node = 'X'
goal_node = 'T'

agent = AStarTraffic(graph, heuristic, start_node, goal_node)
print("\nRunning A* Search with Real-Time Traffic Updates:")
path, cost = agent.a_star()

if path:
    print(f"\nOptimal Path: {path}, Total Travel Time: {cost}")
else:
    print("\nNo valid path found due to traffic congestion.")



Running A* Search with Real-Time Traffic Updates:

Optimal Path: ['X', 'Z', 'U', 'T'], Total Travel Time: 13
